In [ ]:
# Python program to illustrate
# Background subtraction using
# concept of Running Averages
#Source of Code: https://www.geeksforgeeks.org/background-subtraction-in-an-image-using-concept-of-running-average/#
# organize imports
import cv2
import numpy as np

# capture frames from a camera
path = "data/MOV_2564.mp4"
cap = cv2.VideoCapture(path)

# read the frames from the camera
_, img = cap.read()
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# modify the data type
# setting to 32-bit floating point
averageValue1 = np.float32(img)

# loop runs if capturing has been initialized.
while(1):
	# reads frames from a camera
	_, img = cap.read()
	
	img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	# using the cv2.accumulateWeighted() function
	# that updates the running average
	cv2.accumulateWeighted(img, averageValue1, 0.1)
	
	# converting the matrix elements to absolute values
	# and converting the result to 8-bit.
	resultingFrames1 = cv2.convertScaleAbs(averageValue1)

	# Show two output windows
	# the input / original frames window
	cv2.imshow('InputWindow', img)

	# the window showing output of alpha value 0.02
	cv2.imshow('averageValue1', resultingFrames1)
	
	# Wait for Esc key to stop the program
	k = cv2.waitKey(30) & 0xff
	if k == 27:
		break

# Close the window
cap.release()
	
# De-allocate any associated memory usage
cv2.destroyAllWindows()


After adjusting the alpha value 0.1 seems to be a pretty good fit for reducing the movement at the beginning and focusing on the general movement of the gas leak.

Using Optical flow with running average

In [ ]:
# organize imports
import cv2
import numpy as np

# capture frames from a camera
#path = "data/MOV_2564.mp4"
path = "data/MOV_2567.mp4"

cap = cv2.VideoCapture(path)
cap.set(cv2.CAP_PROP_POS_FRAMES, 1800)
# read the frames from the camera
_, img = cap.read()
prev_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# modify the data type
# setting to 32-bit floating point
averageValue1 = np.float32(prev_img)

# loop runs if capturing has been initialized.
while(1):
	# reads frames from a camera
	_, next_img = cap.read()

	next_img = cv2.cvtColor(next_img, cv2.COLOR_BGR2GRAY)
	# using the cv2.accumulateWeighted() function
	# that updates the running average
	
	cv2.accumulateWeighted(next_img, averageValue1, 0.1)

	# converting the matrix elements to absolute values
	# and converting the result to 8-bit.
	resultingFrames1 = cv2.convertScaleAbs(averageValue1)

	flow = cv2.calcOpticalFlowFarneback(prev_img, resultingFrames1, None, 0.5, 3, 9, 3, 5, 1.1, 0)

	horz = flow[..., 0]
	vert = flow[..., 1]
	mag = np.sqrt(horz * horz  + vert* vert)
	# Show two output windows
	# the input / original frames window
	cv2.imshow('Flow Horz', horz)
	cv2.imshow('Flow Vert', vert)

	cv2.imshow('Flow Mag', mag)
	# the window showing output of alpha value 0.02
	cv2.imshow('averageValue1', resultingFrames1)

	# Wait for Esc key to stop the program
	k = cv2.waitKey(30) & 0xff
	if k == 27:
		break
	prv_img = resultingFrames1

# Close the window
cap.release()
	
# De-allocate any associated memory usage
cv2.destroyAllWindows()

After running it with optical flow, it seems a slower rate of change performs worse for optical flow as it can't detect that well the small changes.

Running average with adaptive thresholding

In [ ]:
import cv2
import numpy as np

# Global variables for parameter values
blur_kernel_size = 3
threshold_block_size = 9
threshold_constant = 2
thresh = 0
def preprocessing(frame):
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(frame, (blur_kernel_size, blur_kernel_size), 0)
    #th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, threshold_block_size, threshold_constant)
    _, res = cv2.threshold(blur,thresh,255,cv2.THRESH_BINARY)
    return res


def on_trackbar_blur_kernel_size(val):
    global blur_kernel_size
    blur_kernel_size = max(1, val) if val % 2 != 0 else max(1, val - 1)
    
def on_trackbar_thresh(val):
    global thresh
    thresh = max(0, val)

def on_trackbar_threshold_block_size(val):
    global threshold_block_size
    # Ensure odd value
    threshold_block_size = max(3, val) if val % 2 != 0 else max(3, val - 1)
    

def on_trackbar_threshold_constant(val):
    global threshold_constant
    threshold_constant = val


cap = cv2.VideoCapture('data/MOV_2564.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES, 1800)

# Create windows for displaying frames
cv2.namedWindow('Original Frame')
cv2.namedWindow('Processed Frame')

# Create trackbars
# Blur Kernel Size = BKS ***Bug because it only works with odd numbers since it will break on evens
# Threshold Block Size = TBS
# Threshold Constant = TC
cv2.createTrackbar('BKS', 'Processed Frame', blur_kernel_size, 40, on_trackbar_blur_kernel_size)
cv2.createTrackbar('TBS', 'Processed Frame', threshold_block_size, 20, on_trackbar_threshold_block_size)
cv2.createTrackbar('TC', 'Processed Frame', threshold_constant, 20, on_trackbar_threshold_constant)
cv2.createTrackbar('thresh', 'Processed Frame', thresh, 255, on_trackbar_thresh)

_, og_frame = cap.read()
og_frame = cv2.cvtColor(og_frame, cv2.COLOR_BGR2GRAY)
og_frame = np.float32(og_frame)
while cap.isOpened():
    # Read the current frame
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Check if a frame was successfully captured
    if not ret:
        break
    #removed background
    #cv2.accumulateWeighted(frame, og_frame, 0.1)

    #running_avg = cv2.convertScaleAbs(og_frame)
    # Perform preprocessing on the frame
    preprocessed_frame = preprocessing(frame)
    
    # Display the processed frame
    #cv2.imshow('BG Frame', running_avg)
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Processed Frame', preprocessed_frame)

    # Check if the 'c' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('c'):
        # Move to the next frame
        continue

    # Check if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # Exit the loop if 'q' is pressed
        break

# Release the video capture and close any open windows
cap.release()
cv2.destroyAllWindows()


global thresholding and otsu thresholding wont work because the smoke values are extremely similar to the background and thus the background gets included.

Optical flow with adaptive thresholding

In [ ]:
# organize imports
import cv2
import numpy as np
def preprocessing(frame):
    blur = cv2.GaussianBlur(frame, (3, 3), 0)
    th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 2)
    return th3

# capture frames from a camera
#path = "data/MOV_2564.mp4"
path = "data/MOV_2567.mp4"
cap = cv2.VideoCapture(path)
cap.set(cv2.CAP_PROP_POS_FRAMES, 1800)
# read the frames from the camera
_, img = cap.read()
prev_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
processed_img0 = preprocessing(prev_img)
# loop runs if capturing has been initialized.
while(1):
	# reads frames from a camera
	_, next_img = cap.read()

	next_img = cv2.cvtColor(next_img, cv2.COLOR_BGR2GRAY)
	processed_img1 = preprocessing(next_img)

	flow = cv2.calcOpticalFlowFarneback(processed_img0, processed_img1, None, 0.5, 3, 9, 3, 5, 1.1, 0)

	horz = flow[..., 0]
	vert = flow[..., 1]
	mag = np.sqrt(horz * horz  + vert* vert)
	# Show two output windows
	# the input / original frames window
	cv2.imshow('Flow Horz', horz)
	cv2.imshow('Flow Vert', vert)

	cv2.imshow('Flow Mag', mag)
	cv2.imshow('Regular Img', next_img)
	cv2.imshow('Preprocess Img', processed_img1)
	# Wait for Esc key to stop the program
	k = cv2.waitKey(30) & 0xff
	if k == 27:
		break
	processed_img0 = processed_img1

# Close the window
cap.release()
	
# De-allocate any associated memory usage
cv2.destroyAllWindows()

Other Stuff

In [ ]:
# organize imports
import cv2
import numpy as np

# capture frames from a camera
path = "data/MOV_2564.mp4"
cap = cv2.VideoCapture(path)


# read the frames from the camera
_, img = cap.read()
prev_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# modify the data type
# setting to 32-bit floating point
#averageValue1 = np.float32(prev_img)

# loop runs if capturing has been initialized.
while(1):
	# reads frames from a camera
	_, next_img = cap.read()

	next_img = cv2.cvtColor(next_img, cv2.COLOR_BGR2GRAY)
	# using the cv2.accumulateWeighted() function
	# that updates the running average
	
	#cv2.accumulateWeighted(next_img, averageValue1, 0.7)

	# converting the matrix elements to absolute values
	# and converting the result to 8-bit.
	#resultingFrames1 = cv2.convertScaleAbs(averageValue1)

	flow = cv2.calcOpticalFlowFarneback(prev_img, next_img, None, 0.5, 3, 15, 3, 5, 1.2, 0)

	horz = flow[..., 0]
	vert = flow[..., 1]
	mag = np.sqrt(horz * horz  + vert* vert)
	
	norm_mag = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
	# Show two output windows
	# the input / original frames window
	cv2.imshow('Flow Horz', horz)
	cv2.imshow('Flow Vert', vert)

	cv2.imshow('Flow Mag', mag)
	cv2.imshow('Flow Norm Mag', norm_mag)
	# the window showing output of alpha value 0.02
	cv2.imshow('Input', prev_img)

	# Wait for Esc key to stop the program
	k = cv2.waitKey(30) & 0xff
	if k == 27:
		break
	prev_img = next_img

# Close the window
cap.release()
	
# De-allocate any associated memory usage
cv2.destroyAllWindows()

Note to self: NORMALIZING MATRIX MAKES OPTICAL FLOW MORE NOISEY BUT BETTER CAPTURES THE SMOKE.
Before I normalized between values 0 to 255, but it is way better from 0 to 1 in our case.

In [ ]:
# organize imports
import cv2
import numpy as np

# capture frames from a camera
path = "data/MOV_2564.mp4"
cap = cv2.VideoCapture(path)
cap.set(cv2.CAP_PROP_POS_FRAMES, 1800)
# read the frames from the camera
_, img = cap.read()

prev_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# modify the data type
# setting to 32-bit floating point
#averageValue1 = np.float32(prev_img)

# loop runs if capturing has been initialized.
while(1):
	# reads frames from a camera
	_, next_img = cap.read()

	next_img = cv2.cvtColor(next_img, cv2.COLOR_BGR2GRAY)
	
	# using the cv2.accumulateWeighted() function
	# that updates the running average
	
	#cv2.accumulateWeighted(next_img, averageValue1, 0.7)

	# converting the matrix elements to absolute values
	# and converting the result to 8-bit.
	#resultingFrames1 = cv2.convertScaleAbs(averageValue1)

	flow = cv2.calcOpticalFlowFarneback(prev=prev_img, next=next_img, flow=None, 
				                        pyr_scale=0.5, levels=3, winsize=9, 
					                    iterations=3, poly_n=5, poly_sigma=1.1, flags=0)

	horz = flow[..., 0]
	vert = flow[..., 1]
	mag = np.sqrt(horz * horz  + vert* vert)
	
	norm_mag = cv2.normalize(mag, None, 0, 1, cv2.NORM_MINMAX)
	# Show two output windows
	# the input / original frames window
	cv2.imshow('Flow Horz', horz)
	cv2.imshow('Flow Vert', vert)

	cv2.imshow('Flow Mag', mag)
	cv2.imshow('Flow Norm Mag', norm_mag)
	# the window showing output of alpha value 0.02
	cv2.imshow('Input', prev_img)

	# Wait for Esc key to stop the program
	k = cv2.waitKey(30) & 0xff
	if k == 27:
		break
	prev_img = next_img

# Close the window
cap.release()
	
# De-allocate any associated memory usage
cv2.destroyAllWindows()

Will try moving average background subtraction proposed in original paper:

Link:https://www.sciencedirect.com/science/article/pii/S030626191931685X#s0035

In [ ]:
import cv2 
import numpy as np

def calc_median(frames):
    median_frame = np.median(frames, axis=0).astype(dtype=np.uint8)
    return median_frame

prev_frames = []

path = "data/MOV_2564.mp4"
cap = cv2.VideoCapture(path)
cap.set(cv2.CAP_PROP_POS_FRAMES, 1800)

no_err, frame = cap.read()

prev_limit = 210 #210 was used in the paper linked above

while no_err:

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    prev_frames.append(gray_frame)

    if len(prev_frames) > prev_limit:
        prev_frames.pop(0)

    median_frame = calc_median(prev_frames)

    subtracted_frame = cv2.absdiff(gray_frame, median_frame)

    cv2.imshow('Subtracted Frame', subtracted_frame)
    cv2.imshow('OG Frame', gray_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    no_err, frame = cap.read()
cap.release()
cv2.destroyAllWindows()